In [1]:
from utils.helper_clustering_functions import KMeanClustering ,kmeans_with_smape_ts ,kmeans_with_min_distance
from utils.helper_similarity_metrics import calculate_dtw_distance , calculate_error_metrics ,calculate_cosine_similarity_char ,CoinCrossMappingSimilarity ,smape,smape_distance_metric
from utils.helper_visualization_functions import plot_and_save , cluster_visualization_of_time_series
import pandas as pd
import numpy as np
import os 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import csv


In [2]:
import csv

directory_names = {
    "preprocessed_data_dir_name":"ProcessedData",
    "visualization_data_dir_name":"VisualizationData",
    "testing_garbage_dir_name" :"TestingGarbage",
    "ResultsDirectory":"SimilarityResults",
    "cluster_dir_path":"ClusterResultsVisualization"
}
for key , value in directory_names.items():
    print(f"Creating Directories : {value}")
    os.makedirs(directory_names[key], exist_ok=True)
    
files_path = {
    'raw_price_data' : os.path.join("Datasets","raw_datasets","prices.csv"),
    'raw_token_names' : os.path.join("Datasets","raw_datasets","token_names.csv"),
    "token_names":os.path.join("TestingGarbage","token_names.csv"),
    "similarity_results_file_path":os.path.join("SimilarityResults","similarity_results_version_0.1.csv")
}

Creating Directories : ProcessedData
Creating Directories : VisualizationData
Creating Directories : TestingGarbage
Creating Directories : SimilarityResults
Creating Directories : ClusterResultsVisualization


# Helping function

In [ ]:


if __name__=="__main__":
    # step-1 Reading price data 
    cols_to_ignore = ['Unnamed: 0']
    raw_price_df = pd.read_csv(
                        files_path['raw_price_data'],
                        compression='gzip',
                        usecols=lambda col: col not in cols_to_ignore)
    
    # raw_price_df =  raw_price_df.head(1000*20)
    # step-2 Reading Token data
    token_names_df = pd.read_csv(files_path['raw_token_names'] )

    token_names_df = token_names_df.head(1000)
    number_of_unique_token =  len(token_names_df['id'].unique())
    
    print(f"Shape of price data : {raw_price_df.shape}")
    print(f"shap of token names data : {token_names_df.shape}")
    print(f"Number of Unique Token : {number_of_unique_token}")

    # step-3 Merging the data
    merged_df = raw_price_df.merge(token_names_df, left_on=['network_id', 'base_currency'], right_on=['network_id', 'id'])
    # merged_df = raw_price_df.merge(token_names_df, left_on=['base_currency'], right_on=['id'])
    
    number_of_unique_token_after_merging =  len(merged_df['base_currency'].unique())
    print(f"Number of Unique Token after merging : {number_of_unique_token_after_merging}")

    merged_df['token_id'] = merged_df['base_currency'].astype(str) + '_' + merged_df['network_id'].astype(str)

    selective_base_currency = list(merged_df['base_currency'].unique())[:20]

    merged_df_filtered = merged_df[ merged_df['base_currency'].isin(selective_base_currency)  ]

    price_pivot = merged_df_filtered.pivot_table(index='timestamp_utc', columns='token_id', values='open')
    price_pivot = price_pivot.dropna()

    scaler = StandardScaler()
    price_scaled = scaler.fit_transform(price_pivot.T)  # Transpose so each row is a token


    # labels = KMeanClustering(n_clusters=10,
    #                 price_scaled=price_scaled,
    #                metric = "dtw",
    #                 max_iter=50)
    
    # cluster_results = pd.DataFrame({'token_id': price_pivot.columns, 'cluster': labels})

    # results_with_cluster_id = pd.merge(merged_df_filtered,cluster_results,on='token_id')

    # results_with_cluster_id['timestamp_utc'] = pd.to_datetime(results_with_cluster_id['timestamp_utc'])
    # results_with_cluster_id = results_with_cluster_id.sort_values(by='timestamp_utc')

    # cluster_visualization_of_time_series(results_with_cluster_id=results_with_cluster_id,cluster_dir_path = directory_names['cluster_dir_path'])


        
    # Perform DBSCAN clustering
    dbscan = DBSCAN(eps=100, min_samples=3, metric=smape_distance_metric)
    
    labels = dbscan.fit_predict(price_pivot.T.values)
    
    cluster_results = pd.DataFrame({'token_id': price_pivot.columns, 'cluster': labels})
    
    results_with_cluster_id = pd.merge(merged_df_filtered,cluster_results,on='token_id')
    
    results_with_cluster_id['timestamp_utc'] = pd.to_datetime(results_with_cluster_id['timestamp_utc'])
    results_with_cluster_id = results_with_cluster_id.sort_values(by='timestamp_utc')
    
    cluster_visualization_of_time_series(results_with_cluster_id=results_with_cluster_id,cluster_dir_path = directory_names['cluster_dir_path'])
    
    price_pivot_df = results_with_cluster_id.pivot_table(index='timestamp_utc', columns='token_id', values='open')


    total_detected_similar_tokens = CoinCrossMappingSimilarity(results_with_cluster_id=results_with_cluster_id, price_pivot_df=price_pivot_df,
                                                               files_path = files_path,
                                                               directory_names=directory_names)

    

,network_id,base_currency,timestamp_utc,open,id,display_name,full_name,token_id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
19995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
